# 2024 MLB Pitch Data - Merging, Cleaning, and Processing


In this notebook, we begin by reading in data which was accessed from Baseball Savant ([baseballsavant.mlb.com](https://baseballsavant.mlb.com)) using the `PyBaseball` package ([PyBaseball GitHub](https://github.com/jldbc/pybaseball/blob/master/docs/statcast.md)) in the `2024 MLB Pitch Data - Month by Month Queries using Pybaseball` notebook and exported into `2024month1.csv, 2024month2.csv, 2024month3.csv, 2024month4.csv, 2024month5.csv, 2024month6.csv`.

After merging that data into a single data frame, we proceed to clean, process, and export our data.

## Importing Necessary Packages

We begin by importing `pandas`.

In [3]:
import pandas as pd

pd.set_option('display.max_columns', None)

## Reading In Data

Next, we read in the data obtained in `2024 MLB Pitch Data - Month by Month Queries using Pybaseball` notebook and merge these data frames together.

In [5]:
month1 = pd.read_csv("2024month1.csv")
month2 = pd.read_csv("2024month2.csv")
month3 = pd.read_csv("2024month3.csv")
month4 = pd.read_csv("2024month4.csv")
month5 = pd.read_csv("2024month5.csv")
month6 = pd.read_csv("2024month6.csv")

In [6]:
month_list = [month1, month2, month3, month4, month5, month6]

full_season = pd.concat(month_list, ignore_index=True)

## Viewing the Dataframe

Having concatenated each month's data into a single dataframe, we now view the top 10 rows of the dataframe.

In [8]:
full_season.head(3)

,Unnamed: 0,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,fielder_2,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,estimated_slg_using_speedangle,delta_pitcher_run_exp,hyper_speed,home_score_diff,bat_score_diff,home_win_exp,bat_win_exp,age_pit_legacy,age_bat_legacy,age_pit,age_bat,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle
0,2781,CH,2024-04-30,81.2,-2.75,5.42,"Crismatt, Nabil",572233,622503,home_run,hit_into_play,NaN,NaN,NaN,NaN,13.0,Christian Walker homers (7) on a fly ball to l...,R,R,R,AZ,LAD,X,NaN,fly_ball,1,1,2024,-0.77,0.19,-0.90,2.31,NaN,606466.0,NaN,0,10,Bot,5.58,73.43,NaN,NaN,NaN,NaN,5.685636,-118.191744,-0.712121,-8.322241,22.270713,-30.482585,3.44,1.56,426.0,109.3,25.0,82.2,1677.0,6.9,747210,669257,518692,666158,571970,605141,571771,681624,606192,53.61,0.98,1.978,2.0,1.0,0.0,3.0,6.0,81,3,Changeup,2,3,2,3,3,4,4,3,Standard,Standard,248.0,0.562,1.381,77.7,7.6,3.839,-1.381,109.3,-1,-1,0.438,0.438,29,33,30,33,1,4,2.0,1.0,12.0,1.0,3.26,0.77,0.77,30.6
1,2853,CH,2024-04-30,81.4,-2.73,5.47,"Crismatt, Nabil",572233,622503,NaN,ball,NaN,NaN,NaN,NaN,14.0,Christian Walker homers (7) on a fly ball to l...,R,R,R,AZ,LAD,B,NaN,NaN,0,1,2024,-0.43,0.49,0.01,1.44,NaN,606466.0,NaN,0,10,Bot,NaN,NaN,NaN,NaN,NaN,NaN,7.032661,-118.366360,-3.434234,-5.333467,21.633209,-27.078275,3.44,1.59,NaN,NaN,NaN,82.3,1707.0,6.8,747210,669257,518692,666158,571970,605141,571771,681624,606192,53.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81,2,Changeup,2,3,2,3,3,2,2,3,Standard,Standard,243.0,0.000,0.049,NaN,NaN,NaN,-0.049,NaN,-1,-1,0.438,0.438,29,33,30,33,1,4,2.0,1.0,12.0,1.0,2.94,0.43,0.43,30.0
2,3007,CH,2024-04-30,82.0,-2.63,5.60,"Crismatt, Nabil",572233,622503,NaN,called_strike,NaN,NaN,NaN,NaN,5.0,Christian Walker homers (7) on a fly ball to l...,R,R,R,AZ,LAD,S,NaN,NaN,0,0,2024,-0.68,0.33,0.25,2.38,NaN,606466.0,NaN,0,10,Bot,NaN,NaN,NaN,NaN,NaN,NaN,7.905751,-119.291548,-1.429611,-7.905534,21.282222,-28.900801,3.44,1.56,NaN,NaN,NaN,83.1,1670.0,6.9,747210,669257,518692,666158,571970,605141,571771,681624,606192,53.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81,1,Changeup,2,3,2,3,3,2,2,3,Standard,Standard,245.0,0.000,-0.047,NaN,NaN,NaN,0.047,NaN,-1,-1,0.438,0.438,29,33,30,33,1,4,2.0,1.0,12.0,1.0,3.04,0.68,0.68,34.1


## Restricting Features I

We are only interested in trying to use our data to classify a pitch's type based on the available data. As such, many of the features currently present are unnecessary for our purposes and thus we restrict to the relevant features.

Note that some features initially seem relevant, but are not. This includes `spin_dir`, `spin_rate_deprecated`, `break_angle_deprecated`,
and `break_length_deprecated`, which have been replaced by `release_spin_rate`, `spin_axis`, `pfx_x`, and `pfx_z`.

Additionally, there are different levels of relevance to remaining features. For example, Logan Webb's four seam fastball has an average velocity of 92.6 MPH, whereas his sweeper has an average velocity of 83.8 MPH - velocity is inherently tied to pitch type and thus can be used to distinguish a pitcher's pitch types in isolation. 

However, certain pieces of information are not intrinsic to a pitch itself, but rather how an individual pitcher uses a specific pitch. Continuing with the example above, Logan Webb threw 83 cutters in the 2024 regular season and all of them were thrown to left-handed hitters. This isn't intrinsic to the motion of the pitch - Webb's cutter is a cutter whether he throws it to a left-handed hitter, a right-handed batter, or with no batter. That being said, understanding how pitchers use their pitches can help us correctly classify a pitch in a game setting. 

For our purposes, it suffices to consider only the factors intrinsic to the pitch itself.

In [10]:
# uncomment the code below to get see the full list of columns

# all_cols = full_season.columns.to_list()
# all_cols

In [11]:
intrinsic_features = ['pitch_type', 'pitch_name', 'pitcher', 'p_throws', 'release_speed', 'effective_speed', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z',
                      'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'release_spin_rate', 'spin_axis', 'arm_angle'
    ]

We now restrict to the features in `join_cols`, which removes all features which are entirely irrelevant.

In [13]:
full_season_restricted_features = full_season[intrinsic_features].copy()

In [14]:
full_season_restricted_features.head(5)

,pitch_type,pitch_name,pitcher,p_throws,release_speed,effective_speed,pfx_x,pfx_z,plate_x,plate_z,vx0,vy0,vz0,ax,ay,az,release_spin_rate,spin_axis,arm_angle
0,CH,Changeup,622503,R,81.2,82.2,-0.77,0.19,-0.90,2.31,5.685636,-118.191744,-0.712121,-8.322241,22.270713,-30.482585,1677.0,248.0,30.6
1,CH,Changeup,622503,R,81.4,82.3,-0.43,0.49,0.01,1.44,7.032661,-118.366360,-3.434234,-5.333467,21.633209,-27.078275,1707.0,243.0,30.0
2,CH,Changeup,622503,R,82.0,83.1,-0.68,0.33,0.25,2.38,7.905751,-119.291548,-1.429611,-7.905534,21.282222,-28.900801,1670.0,245.0,34.1
3,SL,Slider,543518,R,87.8,87.1,0.36,0.32,-0.17,2.96,2.625728,-127.965829,-1.993416,3.386311,25.847088,-28.488175,2658.0,141.0,46.3
4,FF,4-Seam Fastball,543518,R,93.6,92.7,-0.66,1.61,-1.62,3.34,1.121846,-136.228171,-4.695673,-8.446417,30.353319,-11.343094,2403.0,207.0,45.2


Next, we check for null values. Of the 743,664 rows in the data frame, we see that every feature has at least 716,506, with most over 735,000.

In [16]:
#print("The data frame `full_season_restricted' has " + str(full_season_restricted_features.shape[0]) + " rows.")
#print()

#for feature in all_relevant_features:
#    print(feature)
#    print(full_season_restricted_features[feature].info())
#    print()

We now restrict to rows with non-null values.

In [18]:
non_null_restricted_features = full_season_restricted_features[
    full_season_restricted_features['pitch_type'].notnull() &
    full_season_restricted_features['pitch_name'].notnull() &
    full_season_restricted_features['release_speed'].notnull() &
    full_season_restricted_features['pfx_x'].notnull() &
    full_season_restricted_features['pfx_z'].notnull() &
    full_season_restricted_features['plate_x'].notnull() &
    full_season_restricted_features['plate_z'].notnull() &
    full_season_restricted_features['p_throws'].notnull() &
    full_season_restricted_features['vx0'].notnull() &
    full_season_restricted_features['vy0'].notnull() &
    full_season_restricted_features['vz0'].notnull() &
    full_season_restricted_features['ax'].notnull() &
    full_season_restricted_features['ay'].notnull() &
    full_season_restricted_features['az'].notnull() &
    full_season_restricted_features['effective_speed'].notnull() &
    full_season_restricted_features['release_spin_rate'].notnull() &
    full_season_restricted_features['pitcher'].notnull() &
    full_season_restricted_features['spin_axis'].notnull() &
    full_season_restricted_features['arm_angle'].notnull()
    ].copy()

As can be verified below, `non_null_restricted_features` has 712,863 rows - each with non-null values.

In [20]:
#print("The data frame `full_season_restricted' has " + str(non_null_restricted_features.shape[0]) + " rows.")
#print()

#for feature in intrinsic_features:
#    print(feature)
#    print(non_null_restricted_features[feature].info())
#    print()

## Minor Feature Engineering

### `pitch_type` and `pitch_name`; A Binary Version of `p_throws`

Note that `pitch_type` is just an abbreviation of `pitch_name`; ultimately, we will only retain `pitch_name`. However, we first observe the distribution of the different pitch types.

In [23]:
non_null_restricted_features['pitch_name'].value_counts(normalize=True)

4-Seam Fastball    0.321041
Slider             0.165077
Sinker             0.153988
Changeup           0.108809
Cutter             0.080027
Curveball          0.067089
Sweeper            0.053630
Split-Finger       0.024169
Knuckle Curve      0.019095
Slurve             0.003120
Other              0.001491
Forkball           0.001020
Eephus             0.000777
Knuckleball        0.000450
Screwball          0.000086
Pitch Out          0.000067
Slow Curve         0.000066
Name: pitch_name, dtype: float64

In [24]:
non_null_restricted_features['pitch_type'].value_counts()

FF    228858
SL    117677
SI    109772
CH     77566
FC     57048
CU     47825
ST     38231
FS     17229
KC     13612
SV      2224
FA      1063
FO       727
EP       554
KN       321
SC        61
PO        48
CS        47
Name: pitch_type, dtype: int64

Given the extreme rarity of some pitches, we will widen the class `Other` to include `Slurve`, `Forkball`, `Eephus`, `Knuckleball`, `Screwball`, `Pitch Out`, and `Slow Curve`. We will considered models which use this expanded `Other` class and ones which drop the class entirely.

We also record the `pitch_name` classes as integers between 0 and 9 and create a binary version of the `p_throws` feature.

In [26]:
p_throws_binary = []
pitch_name_int = []

for counter in non_null_restricted_features.index:
    if non_null_restricted_features.at[counter, 'pitch_type'] == 'FF':
        pitch_name_int.append(0)
    elif non_null_restricted_features.at[counter, 'pitch_type'] == 'SL':
        pitch_name_int.append(1)
    elif non_null_restricted_features.at[counter, 'pitch_type'] == 'SI':
        pitch_name_int.append(2)
    elif non_null_restricted_features.at[counter, 'pitch_type'] == 'CH':
        pitch_name_int.append(3)
    elif non_null_restricted_features.at[counter, 'pitch_type'] == 'FC':
        pitch_name_int.append(4)
    elif non_null_restricted_features.at[counter, 'pitch_type'] == 'CU':
        pitch_name_int.append(5)
    elif non_null_restricted_features.at[counter, 'pitch_type'] == 'ST':
        pitch_name_int.append(6)
    elif non_null_restricted_features.at[counter, 'pitch_type'] == 'FS':
        pitch_name_int.append(7)
    elif non_null_restricted_features.at[counter, 'pitch_type'] == 'KC':
        pitch_name_int.append(8)
    else:
        non_null_restricted_features.at[counter, 'pitch_type'] = 'OT'
        non_null_restricted_features.at[counter, 'pitch_name'] = 'Other'
        pitch_name_int.append(9)
    if non_null_restricted_features.at[counter, 'p_throws'] == 'R':
        p_throws_binary.append(0)
    else:
        p_throws_binary.append(1)

non_null_restricted_features.insert(1, 'pitch_name_int', pitch_name_int)
non_null_restricted_features.insert(5, 'p_throws_binary', p_throws_binary)

We can now verify the remaining classes and their relative value counts.

In [28]:
non_null_restricted_features['pitch_name'].value_counts(normalize=False)

4-Seam Fastball    228858
Slider             117677
Sinker             109772
Changeup            77566
Cutter              57048
Curveball           47825
Sweeper             38231
Split-Finger        17229
Knuckle Curve       13612
Other                5045
Name: pitch_name, dtype: int64

## Exporting the Data Frame

We now export the data frame with `pitch_type` removed and `p_throws_binary` included.

In [31]:
intrinsic_features.remove('pitch_type')
intrinsic_features.insert(1,'pitch_name_int')
intrinsic_features.insert(4,'p_throws_binary')

intrinsic_features = non_null_restricted_features[intrinsic_features].copy()

In [32]:
intrinsic_features.head()

,pitch_name,pitch_name_int,pitcher,p_throws,p_throws_binary,release_speed,effective_speed,pfx_x,pfx_z,plate_x,plate_z,vx0,vy0,vz0,ax,ay,az,release_spin_rate,spin_axis,arm_angle
0,Changeup,3,622503,R,0,81.2,82.2,-0.77,0.19,-0.90,2.31,5.685636,-118.191744,-0.712121,-8.322241,22.270713,-30.482585,1677.0,248.0,30.6
1,Changeup,3,622503,R,0,81.4,82.3,-0.43,0.49,0.01,1.44,7.032661,-118.366360,-3.434234,-5.333467,21.633209,-27.078275,1707.0,243.0,30.0
2,Changeup,3,622503,R,0,82.0,83.1,-0.68,0.33,0.25,2.38,7.905751,-119.291548,-1.429611,-7.905534,21.282222,-28.900801,1670.0,245.0,34.1
3,Slider,1,543518,R,0,87.8,87.1,0.36,0.32,-0.17,2.96,2.625728,-127.965829,-1.993416,3.386311,25.847088,-28.488175,2658.0,141.0,46.3
4,4-Seam Fastball,0,543518,R,0,93.6,92.7,-0.66,1.61,-1.62,3.34,1.121846,-136.228171,-4.695673,-8.446417,30.353319,-11.343094,2403.0,207.0,45.2


Finally, we export these data frames as CSVs to be used in the main notebook.

In [34]:
intrinsic_features.to_csv('intrinsic_features.csv', index=False)